# Introduction: 

Our software will solve the problem of Forward Automatic Differentiation and Reverse Automatic Differentiation. Both of these methods allow for the accurate and easy gradient calculation for a given function. However, they differ in the way that the gradient is calculated. Forward Automatic Differentiation requires one pass moving ''forward'' through the computational graph. It uses the chain rule and dual numbers to calculate the derivative as we go. In contrast, reverse mode first constructs the computational graph and partial derivatives in the forward pass. Then, it propagates through the computational graph with a reverse pass to compute the gradient with respect to the nodes in the graph. 

There are many areas in which Automatic Differentiation is necessary such as gradient calculation in neural networks (reverse mode is typically used), Jacobian calculation, gradient calculation, finding the local maxima and minima of functions (Newton's method), and optimizing them. With automatic differentiation, it is possible to calculate a derivative to machine precision in a computationally efficient manner. This method allows us to avoid the burden of complexity from symbolic differentiation while also having better accuracy than numerical differentiation.

# Background:

### Chain Rule

The chain rule allows us to break down a composite function into elementary operations, particularly when calculating the derivative of a function. It is described like so:
${dx}=\frac{df}{dg}\frac{dg}{dx}$

### Elementary operations
In forward automatic differentiation, a given function is first broken down into elementary functions. Elementary functions include multiplication, addition, subtraction, division, and other basic functions. Once a given function is "divided" into the elementary functions calculated at each step within the function, a graph can be generated where each node is a specific stage in the calculation and each edge is an elementary operation applied to a given node. Applying the derivative chain rule in this graph makes it possible to calculate the gradients and the "contributions" of a given node. 

### Primal and tangent traces
When examining this broken-down graph, the primal trace and tangent trace allow us to calculate the intermediate results at a given step in our differentiation calculation. The forward primal trace of a function computes the value of each intermediate variable $v_j$ at step j while the tangent trace computes the derivatives of these intermediate values, $D_pv_j$ at a given step j. 
### Seed Vector
In order to actually calculate a specific derivative at different steps, we use a seed vector. A seed vector is a "direction" vector that allows us to evaluate a derivative with the weighted combination of the seed vector. For example, for a function $f(x_1, x_2)$, we could calculate $\frac{df}{dx_1}$ with a seed vector of $ p = [1,0]$ or $\frac{df}{dx_2}$ with a seed vector of $ p = [0, 1]$. 
### Example 
The following table displays a simple example of calculating the forward primal trace, tangent trace, and values with seed vectors, from a lab completed in class. The equation $f(x_1, x_2) =  e^{-(sin(x_1)- cos(x_2))^2}$ and we evaluate $f(\frac{\pi}{2}, \frac{\pi}{3})$:


  Forward Primal Trace   | Forward Tangent Trace |  p = $[1,0]^T$    | p = $[0,1]^T$
  -----------------------|-----------------------|-------------------|---------------
  v0 = x_1 = pi / 2      | Dpv0 = p1             | 1                 | 0
  v1 = y_1 =  pi / 3     | Dpv1 = p2             | 0                 | 1
 | | | 
  v_2 = sin(v0) = 1      | Dpv2 = cos(v0)*Dpv0   | 0                 | 0
  v_3 = cos(v1) = 1/2    | Dpv2 = -sin(v1)*Dpv1  | 0                 | -sqrt(3) / 2
 | | | 
  v4 = v2 - v3 = 1/2    | Dpv4 = Dpv2 - Dpv3    | 0                 | sqrt(3) / 2
 | | | 
  v5 = v4 * v4 = 1/2     | Dpv5 = 2*v4 *Dpv4     | 0                 | sqrt(3) / 2
 | | | 
  v_6 = -1 * v_5 = -1/4  | Dpv6 = -Dpv5          |  0                | -sqrt(3) / 2
 | | | 
  v_7 = e^v6 = e^(-1/4)  | Dpv7 = Dpv6 * e^v6    | 0                 | (-sqrt(3) / 2) * e ^(-1/4) 

### Dual Numbers
Dual numbers are another important concept that is extremely useful in forward automatic differentiation. A dual number consists of a real and a dual part (much like complex numbers consist of a real and imaginary part) where $z = a + b\epsilon$ where a is the real part and b the dual. Dual numbers have notable addition and multiplication properties where, if $z_1 = a_1 + b_1\epsilon$ and $z_2 = a_2 + b_2\epsilon$, $z_1 + z_2 = (a_1 + a_2) + (b_1 + b_2)\epsilon$ and $z_1 * z_2 = (a_1 * a_2) + (a_1b_1 + a_2b_2)\epsilon$. With respect to automatic differentiation, a dual number can represent a real part $a = f(x)$ and a dual part where $b = f'(x)$. With respect to traces, this real part can represent the primal trace and the dual part the tangent trace. This makes it easy to calculate the derivative and function at a given step, because the addition and multiplication properties of the dual numbers as discussed above correctly uphold the chain rule, as proven in the lecture with Taylor series expansion. 

### Jacobian 

It is often necessary to compute and evaluate derivatives of a function at a given point in order to correctly determine the Jacobian (defined below); thus, forward automatic differentiation can be crucial to such calculations. 

The Jacobian is a matrix of first-order partial derivatives of a given function with respect to the dependent variables. It is often crucial when calculating things such as Newton's method. As an example, take two example functions $x = u^2 - v^2$ and $y = u^2 + v^ 2$. For the Jacobian with respect to this system, we would want: 

$\begin{array}{cc}
\frac{dx}{du} & \frac{dx}{dv} \\
\frac{dy}{du} & \frac{dy}{dv} \\
\end{array}$  

Which would yield: 

$\begin{array}{cc}
2u & -2v \\
2u & 2v \\
\end{array}$

## Reverse Mode 

In contrast to the above description of forward mode, in which the chain rule is used to calculate the gradient in one forward pass, the reverse mode does so with a backward pass through the computational graph. It ultimately features two passes with the first being a construction of the computational graph and partial derivatives. Then there is reverse propagation through this computational graph that recovers the computed partial derivative at each step with respect to the intermediate variable at a given step. The adjoint is defined as $\bar{v}_{j-m} = \frac{\partial f_i}{\partial v_{j-m}}$ where $v_{j-m}$ is the intermediate variable at the give computational step, and $f_i$ is defined as the ith output in the computational calculation. The adjoint is a good representation of the sensitivity f the output with respect to this intermediate $v_{j-m}$. Using this adjoint and sensitivity measure, we are able to compute the gradient of f; because each of the adjoints of the computational graph are representations of the sensitivity, we can compute the gradient $\nabla f_i$ with the first m adjoints are the first m components of the gradient. Thus, by calculating the adjoint and the intermediate value itself $v_{j-m}$, the gradient can be reconstructed.

Reverse mode mimics the recreation of the chain rule that we used in forward mode to calculate the derivative and gradient in forward mode. It first computes each of the partial derivatives $\frac{\partial v_j}{\partial v_i}$. We then calculate the adjoints and the sensitivity measure with a reverse pass to accumulate the computational contributions of the children intermediate steps of the given position. Thus, the adjoint is calculated with $\bar{v_j} = \bar{v_i} + \bar{v_j}\frac{\partial v_j}{\partial v_i}$ and thus we are able to calculate the overall gradient using this recursive calculation and reverse propagation through the graph. 

Because reverse mode features a forward computation of the partial derivatives followed by a backward propagation of the computational graph while forward mode is simply the calculation of the derivative and the value at each step via the chain rule, if there are significantly more outputs than inputs, forward mode is more efficient, whereas more outputs means that reverse mode is more efficient. Reverse mode also features a space tradeoff between forward AD: the computational graph of the function is necessary for the reverse mode gradient calculation, whereas this is not necessary for forward AD calculation. 

## Additional Notes
Our group thinks that 3Blue1Brown does a great exploration of the topic of automatic differentiation: https://www.youtube.com/watch?v=tIeHLnjs5U8

Here is an image from the video which shows a computation graph that represent the operations in a trivial NN: 
<div>
<img src="https://www.3blue1brown.com/content/lessons/2017/backpropagation-calculus/tree-extended.png" width="400">
</div>

# How to Use Our Package:

Our code is deployed as a package via PyPI. Thus it can be downloaded with pip. The following commands should successfully load our package.

```
python -m pip install --upgrade pip
pip install -r requirements.txt
pip install -i https://test.pypi.org/simple/ team48-autodiff-package
```
After this, you should be able to use our given package with both reverse or forward mode calculation of the gradient. 

For the forward mode case, you must declare the AutoDiff() class object with the function or functions defined as entry parameters. From there, you can calculate the derivative at a specific x entry value by passing your x value to the specified class object. There is also an optional seed vector input value p. The below example shows different ways to use our package. It is important to note that in the gradient, the outputs of the function are largely dependent on the number of variables (you can see the different types of variables output as below). We also provide a gradient() (returns the gradient) and evaluate() (returns the actual value) functions that can be called via AutoDiffObject.gradient(x) and AutoDiffObject.evaluate(x). It is also important to note that in our implementation, if there is more than one variable passed to the AutoDiff class, the seed vector (p) must be specified accordingly. Below are simple examples of use case:

```
from team48_autodiff_package.autoDiff.forward.AutoDiff import *
from team48_autodiff_package.autoDiff.forward.single_var_forward import *
from team48_autodiff_package.autoDiff.forward.single_var_forward import *

d = 26

# Single function single value (vector or scalar accepted)
forward_diff_outs = AutoDiff(lambda x: exp(x**2))
print(forward_diff_outs(d)) # Returns the derivative of the function

#String evaluation
func = 'x**2 + y'
ad_instance = AutoDiff(func)
print(ad_instance([10,10], 0))
```

Reverse mode can be called with the gradient() function in autoDiff/reverse/reverse.py. Inputs to the function must be declared as Variable objects so that the graph knows to use them for the derivative. Variable objects are declared with a numerical value and a string name. Reverse mode functions can also be submitted via string for convenience.

```
single variable reverse mode call:

func = lambda x: 2**x

x = Variable(5,'x')
derivatives = gradient(func(x))
print(derivatives['x'])
```

In our package, there are a few modules that can be imported for usage. We recommend using the syntax in the example above ''from team48_autodiff_package.AutoDiff import *''. This will allow direct calling and integration of special functions (cos, asin, log, and the others specified in the implementation description without needing to specify the parent class. Thus, this allows for smooth integration. 

Our additional feature is reverse mode. Thus, we will also specify how to run this additional feature in our code. The reverse mode implementation has a function gradient that, given a root operation node as input, from a previously specified function f, will calculate the gradient of the function using an iterative Breadth-First-Search through the graph. The final gradient is returned as a dictionary that specifies the gradient for a given variable that is the key in the dictionary. Some of these features were not completed by the end of the project, but our progress is still tracked via the code and comments.

# Software Organization:

## Directory Structure and Basic Modules:
Our directory follows the below structure. Because we have started integrating the package into PyPI, our software has some additional files to make this possible. 

```
team48/
├── docs
│   └── documentation
├── LICENSE
├── README.md
├── requirements.txt
├── pyproject.toml
├── tests
└── src 
    ├── team48_autodiff_package
    |   ├── autoDiff
        |   ├── reverse
        |   └── forward
    |   ├──dualNumber
    |   └──nodeGraph
    └── team48_autodiff_package.egg-info
```

Inside src, we have a folder named team48_autodiff_package which includes most of our central modules. These are divided into the following folders and subfolders:
  - autoDiff:
    - forward:
      - AutoDiff.py: contains the AutoDiff class that returns the derivative value for a value and function as specified in the example in the 'How to Use' section above.
      - forward.py: helper functions for testing and AutoDiff functions enabling multiple inputs
      - single_var_forward: contains the code for computing the derivative given a single input
    - reverse:
      - grad_ops.py: helper functions to allow calculations between gradients
      - reverse_mode_helpers.py: helper functions for testing multi-variable approximation
      - reverse.py: contains gradient calculation using reverse mode and supporting helpers
    - dualNumber:
        - dual.py: contains the dual class and the overloaded functions for the DualNumber class
    - nodeGraph: 
      - node_operation.py: file containing the correct implementation of operations between nodes given the data structure setup
      - node.py: declares Node, Operation, Constant, NodeQueue classes and overloaded operations for the data type
## Tests and Running
Our tests for the code are in the tests folder. All of the tests can be run with the pytest command. These are integrated into our workflow using GitHub workflows. We have 4 files, test_dual.py, test_forward_ad.py, test_reverse_ad.py, and test_ad.py that test each of the corresponding files in src/team48_autodiff_package. To run these tests, ensure that you are in the root directory and have installed the necessary dependencies and packages (this is in the how to use section). Then use `pytest` to run the tests. Our tests cover 94% of the code in src/team48_autodiff_package. 

Our package can be installed as specified in the `How to Use Our Package` section.

# Implementation:
Below are the two classes that we implemented in our package. The Dual Numbers class serves not only as a core class, but also as a core data structure necessary for effective forward automatic differentiation. Details about the classes are specified below.
- **DualNumber class**:
  - Attributes: 
    - real: a real number part of type float or int
    - dual: a dual number part of type float or int
  - Methods:
    - We overloaded some of the following elementary operations to correctly work for dual numbers as abstractly explained in the background section. The derivative is thus saved as the dual component of the DualNumber. 
    - Overloaded methods:
      - +, -, \, *, **, ==, __neg__ 
      - sin(), cos(), tan()
      - asin(), acos(), atan()
      - exp()
      - log()
      - sqrt()
      - sinh(), cosh(), tanh()
      - logistic()

- **Node class**:
  - Attributes:
    - Inherits from the np.ndarray class
  - Methods:
    - We overloaded some of the following elementary operations to call a wrapper function with each call to the method. This wrapper call allows the correct operation with respect to the value stored at the node. 
    - Overloaded methods:
      - +, -, \, *, ** 

- **Constant class**: 
  - Attributes: 
    - Builds on top of the Node class
  - Methods
    - Overloads the __new__ function to return the specified array type(np.array if specified otherwise ndarray for multi-space) and increments constant count accordingly

- **Variable class**: 
  - Attributes: 
    - Builds on top of the Node class
  - Methods
    - Overloads the __new__ function to return the specified array type(np.array if specified otherwise ndarray for multi-space) and increments variable count accordingly

- **Operation class**: 
  - Attributes: 
    - Builds on top of the Node class
  - Methods
    - Overloads the __new__ function to return a corresponding np.ndarray and initialize a new operation for a node accordingly 

- **NodeQueue class**: 
  - Attributes: 
    - deque of nodes 
    - deque of node ids
  - Methods
  - push: push new node onto the queue
  - pop: pop node from front of the queue
  - __contains__: check if node is in queue
  - __len__ return length of queue

- **AutoDiff class**: 
  - Attributes: 
    - func: An input function associated with the object
    - Methods
      - A given instance of the class can be passed a value (at this time a float or int) and will return the evaluated derivative value at the parameter value; the derivative is returned. 

  Our code also makes use of helper functions that can be found in src/team48_autodiff_package/ad.py. There are 3 functions that serve various testing and implementation purposes that are described below:

- **Functions in single_var_forward.py**:
  - **single_derivative_approximation(fx, x)**:
    - This function accepts a function and a value (integer or float) x and closely approximates the derivative at x using $\frac{f(x) - f(x - h)}{h}$ where $h = 1.e-8$.
    - This is used for testing purposes. 
  -**check_error(test_value, approx)**:
    -This function checks if `test_value` (integer or float) is very close to `approx` (integer or float). It checks if the difference between the two values is smaller than 1e-6*(1 + abs(approx)). If the difference is smaller than the error, True is returned; otherwise False is returned.
    - This is used for testing purposes to compare single_derivative_approximation() against the true derivative calculation with our AutoDiff class to ensure correctness.
  -**single_derivative(fx, x)**
    -This function calculates the precise derivative of a function; x is a DualNumber and fx is a function. The derivative is returned.
    -This function is called in the AutoDiff class __call__ function when an AutoDiff object is passed a value.

- **Functions in forward.py**:
  -**derivative(f, p, values)**: This function calculates the derivative of the input function f with seed vector(s) p and input variables of values. The function works for an array of p values and individual numbers.
  - The function can also work when f is a string and will parse the statement to return for the desired value.  
  - **gradient(f, values)**: return the gradient of a function f for the given input values. Returns as an array.

- **Functions in reverse.py**:
  - **resize_for_node(node, adjoint)**: return the new adjoint value of the node after summing across the adjoint and node dimension.
  -**derivative(f, p, values)**: This function calculates the derivative of the input function f with seed vector(s) p and input variables of values. The function works for an array of p values and individual numbers.
  - The function can also work when f is a string and will parse the statement to return for the desired value.  
  - **gradient(root, p)**: return the gradient of a function f calculated from the root node passed to the function.

- **Functions in reverse_mode_helpers.py**:
  - **def multi_var_approx(fx, values)**: return the approximation value of the function fx with the given input values.

- **Functions in grad_ops.py**:
  - **def multi_var_approx(fx, values)**: return the approximation value of the function fx with the given input values.

- **Content of node_operations.py**:
  - The file contains overloaded special functions to allow them to work for reverse mode passes. The following are overloaded and will correctly calculate in reverse mode:
    - sum
    - log
    - exp
    - sin, cos, tan, 
    - asin, acos, atan
    - sinh, cosh, tanh

-**Functions in reverse_mode_helpers.py**: 
  - **approx(fx, values)**: Approximates the derivative of fx give values; used for testing purposes. 

Finally, there is a hack_solution.py function that enables the coverage CI test integration through github workflows. This checks to ensure that the tests cover greater than 90% of the code. 

All requirements can be found in requirements.txt and can be installed with The implementation of these classes requires the installation of numpy. This should be installed if the directions for installation in a previous section with the command 'pip install -r requirements.txt.'
    
<!-- # Upcoming features:
- **R^m -> R^n mapping**
  - We plan to modify the AutoDiff class to work with functions which accept and return vectors (Numpy array inputs and outputs)
- **Parsing a string into a function**
  - We plan to allow users to submit a function to create an AutoDiff object by passing a string if they so choose, using the eval() functions to parse said string.
- **Higher order derivatives**
  - We plan to implement evaluation of higher order derivatives through an AutoDiff instance by evaluating the derivative multiple times.
## Reverse mode:
  - We plan to implement reverse mode evaluation of the derivative. For this, we will add the following classes:
- **Operation class**:
  - Attributes: 
    - Enum class where all elementary operations are assigned a number  
- **Node class**:
  - Attributes:
    - Optional Parameter Dict
    - operation: an Operation type variable representing the operation at the given node 
    - Value: Dual number value at a given "step" in the calculation
  - Methods:
    - getValue: get the DualNumber value at a given node
    - getOperation: get the given operation at a given node
- **Graph class**:
  - Attributes: 
    - Nodes: nodes in the graph of type node
    - Edges: adjacency list representing edges between different nodes
  - Methods:
    - reverseAutoDiff: backwards traversal of the graph to calculate the derivative with respect to the given values

The core data structures are the classes as discussed above. The Dual numbers will consist of a real number and a dual number (derivative) part. Each node will consist of a operation that occurs at the given node and specific parameters of a dual number type to track the value and derivative in a backwards propagation (we are accounting for reverse mode with this). The graph class will have nodes as discussed above and an adjacency list of edges between these nodes to record the relationships; these will be used to represent our computational graph and are necessary to conduct reverse auto differentiation.

We will also overload more operations for dual numbers by declaring custom versions of these functions in the DualNumber class. We will create our own versions of this and default to numpy versions of the functions. We will be using the numpy array and numpy matrix as fundamental data structures and extend numpy functionality as it allows us to conveniently handle vector input and vector functions. Because handling vector input and numpy data structures are critical to our project we will include the external package numpy. In addition to the functions we already overloaded, we plan on overloading the following functions using numpy:
-Inverse trig functions
-Exponentials (any base)
-Hyperbolic functions
-Logarithms (any base)

NetworkX is an external package that allows us to visualize graphs. We will use this package to visualize large graph structures. We intend to include this package out of convenience for the user, but it is more of a stretch goal.

To handle cases for functions where the input dimensions differ from the output dimensions, we will include checks within nodes to enure that the number of variables and dimensions is staying consistent with our defined function. We have discussed a reverseAutoDiff function above and will include that in our library for the function.

Given we have enough time, we may also implement a layer class for our graph to help visualize neural net propagation more effectively. 
-->
# Broader Impact and inclusivity Statement

  **Broader Impact**
  
  Our software is intended to help people efficiently and conveniently find the derivatives and gradients of functions. Derivative calculators like Wolfram or graphing calculators are widely available, but an automatic differentiation package is particularly powerful because of its O(1) speed and ability to integrate into other programs. Our hope is that individuals will use this software in machine learning, game theory, mathematics, and other applicable fields to contribute meaningful, positive work to society as a whole. Misuses of this package could be as trivial as a high schooler cheating on calculus homework to working on serious ML research with the goal of personal game at the expense of others. As previously mentioned, our package has particular applications in AI, and significant and possible uses of our package could involve the creation of a model that can defeat any human at a game, which could be particularly problematic if an individual with a gambling addiction was pitted against such a model. One such example of this is the AlphaZero engine from chess:

  https://mmsubra1.medium.com/machine-learning-for-chess-alphazero-vs-stockfish-b58638e73fee

  While the latter example is exciting, it can easily be used by humans to cheat in online matches or even in in-person tournaments if a player smuggles in a device. With great computing power comes great responsibility, and we encourage our users to consider the consequences of their potential usage of our package.

  **Software Inclusivity**

  We encourage all people to use this software regardless of age, race, gender, socioeconomic status, national origin, or any other personal characteristics. That being said, we understand that the software is not equally accessible to everyone. For one thing, all of the comments, docstrings, command names, and this documentation file are in english or use english-based naming conventions. As the authors of this package, we are the most comfortable with English and didn’t have the resources or time to write multiple versions of the code. Fortunately, most of the actual interface only requires the user to read mathematical symbols and numbers. While the user will need to translate some minor documentation to understand how the AutoDiff object works, it is fairly straightforward and can be called without significant English knowledge.
<!-- # Future Plans -->

# Future
In the future, we would like to implement higher-order derivatives through an AutoDiff instance by evaluating the derivative multiple times. This would be able to calculate, as the name suggests,  higher order derivatives up to a specified level; for example this would be able to calculate f''(x) for an input function and input value x. 

We would also like to integrate NetworkX visualization into our project for reverse mode computations. NetworkX is a package that allows you to visualize graphs; we think that being able to see the computational graph for reverse mode would be helpful for users and interesting. 

